[] 3.1 Use gensim.downloader.load to load the word2vec-google-news-300 pretrained embedding model.

In [51]:
import gensim.downloader

word2vec_googlenews = gensim.downloader.load('word2vec-google-news-300')


3.2 Use the tokenizer from nltk to extract words from the Reddit posts. Display the number of tokens in the training set.


In [52]:
import json
import gzip
import nltk

jsonfiledirectory = "/Users/rosiers/Documents/GitHub/472-assignment1/goemotions.json.gz"

with gzip.open(jsonfiledirectory, "r") as f:
    data = json.loads(f.read().decode("utf-8"))
strings_token = []
for item in data:
    strings_token.append(nltk.word_tokenize(item[0]))
count = 0
for str in strings_token:
    count = count + len(str)
print(count)

2642128


3.3 Compute the embedding of a Reddit post as the average of the embeddings of its words. If a word has no embedding in Word2Vec, skip it.

In [59]:
embeddings = []
for str in strings_token:
    strings_embedding = []
    for tkn in str:
        if tkn in word2vec_googlenews:
            strings_embedding.append(word2vec_googlenews[tkn])
        else:
            continue
    embd = zip(*strings_embedding)
    average_embedding = []
    for strList in embd:
        avg = sum(strList)/len(strList)
        average_embedding.append(avg)
    embeddings.append(average_embedding)

300


3.4 Compute and display the overall hit rates of the training and test sets (i.e. the % of words in the Reddit posts for which an embedding is found in Word2Vec).

[-0.13867188  0.12597656  0.32617188  0.09960938 -0.23730469  0.01483154
  0.04980469 -0.09179688  0.20507812  0.296875    0.08837891 -0.24609375
  0.10351562  0.01733398 -0.28125     0.359375    0.24804688  0.1015625
 -0.0324707  -0.14355469 -0.12597656  0.21972656  0.16992188 -0.09863281
  0.046875   -0.234375   -0.21289062 -0.05273438 -0.01300049  0.21484375
  0.07666016 -0.08789062  0.02722168  0.03222656  0.21679688 -0.08837891
 -0.34960938  0.27148438  0.17578125  0.22851562  0.21289062  0.12158203
 -0.0168457  -0.24609375 -0.17578125  0.10498047 -0.09765625  0.06494141
  0.265625    0.21972656 -0.16015625  0.07080078  0.04736328 -0.16699219
  0.24023438  0.17773438  0.09179688 -0.16210938  0.01794434  0.10400391
 -0.19042969  0.04907227 -0.1796875  -0.11914062 -0.01055908 -0.16894531
  0.08789062 -0.02478027  0.09082031 -0.19921875  0.11083984 -0.02258301
  0.08642578  0.16699219 -0.12988281  0.06591797 -0.05273438  0.09912109
  0.11328125  0.390625   -0.05395508 -0.09863281 -0.

3.5 Train a Base-MLP: a Multi-Layered Perceptron (neural network.MLPClassifier) with the default parameters.

3.6 Train a Top-MLP: a better performing Multi-Layered Perceptron found with whatever hyperparameters you want.

3.7  Display the performance of your classifiers using metrics.classification report and add these to your performance file.